## ATTENTION

Ce Notebook nécessite des ressources GPU. Pour cela, utilisez l'environnement Google Colab plutôt que Binder :

[Lien vers Google Colab](https://colab.research.google.com/github/lsteffenel/M2Atmo_et_Climat/blob/main/10-Transfer_learning.ipynb)

# Transfer Learning

Jusqu'à présent, nous avons entraîné des modèles précis sur de grands ensembles de données, et nous avons également téléchargé un modèle pré-entraîné que nous avons utilisé sans avoir besoin d'entraînement. Mais que se passe-t-il si nous ne trouvons pas de modèle pré-entraîné qui réponde exactement à vos besoins, et si nous ne disposons pas d'un ensemble de données suffisamment important pour entraîner un modèle à partir de zéro ? Dans ce cas, il existe une technique très utile appelée [apprentissage par transfert] (https://blogs.nvidia.com/blog/2019/02/07/what-is-transfer-learning/).

Avec l'apprentissage par transfert, nous prenons un modèle pré-entraîné et le réentraînons sur une tâche qui a un certain chevauchement avec la tâche d'entraînement d'origine. Une bonne analogie est celle d'un artiste compétent dans un domaine, comme la peinture, qui souhaite apprendre à s'exercer dans un autre domaine, comme le dessin au fusain. On peut imaginer que les compétences acquises en peignant seront très utiles pour apprendre à dessiner au fusain. 

Dans le domaine de l'apprentissage profond, supposons que nous disposions d'un modèle pré-entraîné très performant pour reconnaître différents types de voitures et que nous souhaitions entraîner un modèle à reconnaître des types de motocyclettes. Une grande partie des apprentissages du modèle de voiture serait probablement très utile, par exemple la capacité à reconnaître les phares et les roues. 

L'apprentissage par transfert est particulièrement efficace lorsque nous ne disposons pas d'un ensemble de données vaste et varié. Dans ce cas, un modèle formé à partir de zéro mémoriserait rapidement les données d'apprentissage, mais ne serait pas en mesure de se généraliser à de nouvelles données. Grâce à l'apprentissage par transfert, vous pouvez augmenter vos chances de former un modèle précis et robuste sur un petit ensemble de données.

## Objectifs

* Préparer un modèle pré-entraîné pour l'apprentissage par transfert
* Effectuer l'apprentissage par transfert avec votre propre petit ensemble de données sur un modèle pré-entraîné
* Ajustement du modèle pour une performance encore meilleure

In [ ]:
!wget http://urca.lsteffenel.fr/IntroDL/data.tar.gz -O data.tar.gz
!tar -xvzf data.tar.gz

## A Personalized Doggy Door

Dans notre dernier exercice, nous avons utilisé un modèle [ImageNet] (http://www.image-net.org/) pré-entraîné pour laisser entrer tous les chiens, mais exclure les autres animaux. Dans cet exercice, nous souhaitons créer une porte pour chien qui ne laisse entrer qu'un chien en particulier. Dans ce cas, nous allons créer une niche automatique pour un chien nommé Bo, premier chien des États-Unis entre 2009 et 2017. Les photos de Bo se trouvent dans le dossier `data/presidential_doggy_door`.

Le problème est que le modèle pré-entraîné n'a pas été entraîné à reconnaître ce chien spécifique et que nous ne disposons que de 30 images de Bo. Si nous essayons de former un modèle à partir de zéro en utilisant ces 30 photos, nous serons confrontés à un surajustement et à une mauvaise généralisation. En revanche, si nous partons d'un modèle pré-entraîné capable de détecter les chiens, nous pouvons tirer parti de cet apprentissage pour acquérir une compréhension généralisée de Bo en utilisant notre petit ensemble de données. Nous pouvons utiliser l'apprentissage par transfert pour relever ce défi.

## Téléchargement du modèle pré-entraîné

Les [modèles pré-entraînés ImageNet](https://keras.io/api/applications/vgg/#vgg16-function) sont souvent de bons choix pour l'apprentissage par transfert de la vision par ordinateur, car ils ont appris à classer différents types d'images. Ce faisant, ils ont appris à détecter de nombreux types de [caractéristiques] (https://developers.google.com/machine-learning/glossary#) susceptibles d'être utiles pour la reconnaissance d'images. Comme les modèles ImageNet ont appris à détecter les animaux, y compris les chiens, ils sont particulièrement adaptés à cette tâche d'apprentissage par transfert qu'est la détection de Bo.

Commençons par télécharger le modèle pré-entraîné. Là encore, ce modèle est disponible directement à partir de la bibliothèque Keras. Lors du téléchargement, il y aura une différence importante. La dernière couche d'un modèle ImageNet est une [couche dense] (https://developers.google.com/machine-learning/glossary#dense-layer) de 1000 unités, représentant les 1000 classes possibles de l'ensemble de données. Dans notre cas, nous voulons que la classification soit différente : **est-ce que c'est Bo ou pas ?** 

Parce que nous voulons que la classification soit différente, nous allons supprimer la dernière couche du modèle. Nous pouvons le faire en réglant le drapeau `include_top=False` lors du téléchargement du modèle. Après avoir supprimé cette couche supérieure, nous pouvons ajouter de nouvelles couches qui produiront le type de classification que nous voulons :

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.summary()

## Geler le modèle de base
Avant d'ajouter nos nouvelles couches au [modèle pré-entraîné] (https://developers.google.com/machine-learning/glossary#pre-trained-model), nous devons franchir une étape importante : **figer les couches pré-entraînées du modèle**. 

Cela signifie que lors de l'entraînement, nous ne mettrons pas à jour les couches de base du modèle pré-entraîné. Au lieu de cela, nous ne mettrons à jour que les nouvelles couches que nous ajouterons à la fin pour notre nouvelle classification. Nous figeons les couches initiales parce que nous voulons conserver l'apprentissage réalisé lors de l'entraînement sur l'ensemble de données ImageNet. Si elles étaient "dégelées" à ce stade, nous détruirions probablement ces précieuses informations. Il sera possible de débloquer et d'entraîner ces couches ultérieurement, dans le cadre d'un processus appelé "réglage fin".

Geler les couches de base est aussi simple que de régler la possibilité d'entraînement sur le modèle à `False`.

In [ ]:
base_model.trainable = False

## Addition de nouvelles couches

Nous pouvons maintenant ajouter les nouvelles couches entraînables au modèle pré-entraîné. Elles prendront les caractéristiques des couches pré-entraînées et les transformeront en prédictions sur le nouvel ensemble de données. 

Nous allons ajouter deux couches au modèle: 
* La première sera une couche de mise en commun, comme nous l'avons vu dans notre [réseau neuronal convolutif] précédent (https://developers.google.com/machine-learning/glossary#convolutional_layer). (Si vous souhaitez mieux comprendre le rôle des couches de mise en commun dans les CNN, lisez [cet article de blog détaillé] (https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/#:~:text=A%20pooling%20layer%20is%20a,Convolutional%20Layer)). 
* Nous devons ensuite ajouter notre couche finale, qui classera Bo ou non Bo. Il s'agira d'une couche densément connectée avec une sortie.

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

Jetons un coup d'œil au modèle, maintenant que nous avons combiné le modèle pré-entraîné avec les nouvelles couches.

In [ ]:
model.summary()

Keras nous donne un bon résumé ici, car il montre le modèle pré-entraîné vgg16 comme une seule unité, plutôt que de montrer toutes les couches internes. Il est également intéressant de noter que nous avons de nombreux paramètres non entraînables car nous avons figé le modèle pré-entraîné.

## Compilation du Modèle

Comme dans nos exercices précédents, nous devons compiler le modèle avec les options de pertes et de métriques. Nous devons ici faire des choix différents. Dans les cas précédents, notre problème de classification comportait de nombreuses catégories. Par conséquent, nous avons choisi l'entropie croisée catégorielle pour le calcul de notre perte. 

Dans le cas présent, nous n'avons qu'un problème de classification binaire (**Bo** ou **non Bo**) et nous utiliserons donc [l'entropie croisée binaire] (https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy). Pour plus de détails sur les différences entre les deux, voir [ici] (https://gombru.github.io/2018/05/23/cross_entropy_loss/). Nous utiliserons également la précision binaire au lieu de la précision traditionnelle.

En réglant `from_logits=True`, nous informons à la [fonction de perte] (https://gombru.github.io/2018/05/23/cross_entropy_loss/) que les valeurs de sortie ne sont pas normalisées (par exemple avec softmax).

In [ ]:
# Important to use binary crossentropy and binary accuracy as we now have a binary classification problem
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

## Augmentation des données

Comme nous avons affaire à un très petit ensemble de données (juste 30 images de Bo), il est particulièrement important d'augmenter nos données. 

La procédure d'augmentation de données est une méthode où nous apporterons de petites modifications aux images existantes, ce qui permettra au modèle d'apprendre à partir d'une plus grande variété d'images. Cela l'aidera à apprendre à reconnaître de nouvelles images de Bo au lieu de se contenter de mémoriser les images sur lesquelles il apparaît.

Afin de faire cette augmentation, nous allons utiliser un générateur d'images qui va faire des opérations variées sur les images existantes. Parmi celles que vous utilisez ci-dessous, on trouve : 
* rotation (10 degrées à gauche ou à droite)
* zoom (10%)
* décalage des images (horizontal et vertical, 10% de la taille)
* effet miroir (horizontal seulement)

Ces transformations une fois combinées, permettent de créer plusieurs images différentes qui permettent de généraliser l'entraînement (par exemple, en mettant Bo "hors centre" de l'image).




In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## Chargement des données

Jusqu'à présent, nous avons vu des ensembles de données sous différents formats. Dans l'exercice MNIST, nous avons pu télécharger l'ensemble de données directement à partir de la bibliothèque Keras. 

Pour cet exercice, nous allons charger des images directement à partir de dossiers en utilisant la fonction [`flow_from_directory`](https://keras.io/api/preprocessing/image/) de Keras. Nous avons configuré nos répertoires pour faciliter le processus, car nos étiquettes sont déduites des noms de dossiers. 

Dans le répertoire `data/presidential_doggy_door`, nous avons les répertoires train et validation, qui contiennent chacun des dossiers pour les images de Bo et non Bo. Dans les répertoires not_bo, nous avons des images d'autres chiens et chats, pour apprendre à notre modèle à exclure les autres animaux. N'hésitez pas à explorer les images pour vous faire une idée de notre ensemble de données.

Notez que [flow_from_directory](https://keras.io/api/preprocessing/image/) nous permet également de dimensionner nos images en fonction du modèle : 244x244 pixels avec 3 canaux.

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/presidential_doggy_door/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/presidential_doggy_door/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='binary', 
                                      batch_size=8)

## Entraîner le modèle

Il est temps d'entraîner notre modèle et de voir comment il se comporte. Rappelons que lors de l'utilisation d'un générateur de données, nous devons explicitement définir le nombre de `steps_per_epoch` :

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

## Discussion sur les résultats

La précision de l'entraînement et de la validation que vous avez obtenu doivent être assez élevée. 

C'est un résultat assez impressionnant compte tenu du petit ensemble de données. Frâce aux connaissances transférées du modèle ImageNet, il a pu atteindre une précision élevée et bien se généraliser. Cela signifie qu'il a une très bonne perception de Bo et des animaux domestiques qui ne sont pas Bo.

Si vous avez constaté une certaine fluctuation dans la précision de la validation, ce n'est pas grave non plus. Nous avons une technique pour améliorer notre modèle dans la section suivante.

## Réglage fin du modèle

Maintenant que les nouvelles couches du modèle sont formées, nous avons la possibilité d'appliquer une dernière astuce pour améliorer le modèle, appelée [fine-tuning] (https://developers.google.com/machine-learning/glossary#f). Pour ce faire, nous débloquons l'ensemble du modèle et l'entraînons à nouveau avec un [taux d'apprentissage] très faible (https://developers.google.com/machine-learning/glossary#learning-rate). Ainsi, les couches préentraînées de base prendront de très petites mesures et s'ajusteront légèrement, améliorant ainsi le modèle dans une faible mesure (on aime grapiller des 0.001%).  

Notez qu'il est important de ne procéder à cette étape qu'une fois que le modèle avec les couches gelées a été entièrement entraîné. Les couches de mise en commun et de classification non entraînées que nous avons ajoutées au modèle plus tôt ont été initialisées de manière aléatoire. Cela signifie qu'elles ont dû être mises à jour assez souvent pour classer correctement les images. Par le biais du processus de [rétropropagation] (https://developers.google.com/machine-learning/glossary#backpropagation), des mises à jour initiales importantes dans les dernières couches auraient entraîné des mises à jour potentiellement importantes dans les couches pré-entraînées également. Ces mises à jour auraient détruit ces importantes caractéristiques pré-entraînées. Cependant, maintenant que les dernières couches sont entraînées et ont convergé, toute mise à jour du modèle dans son ensemble sera beaucoup plus petite (en particulier avec un taux d'apprentissage très faible) et ne détruira pas les caractéristiques des couches précédentes.

Essayons de dégeler les couches pré-entraînées, puis d'affiner le modèle :

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)

## Examiner les prédictions

Maintenant que nous avons un modèle bien entraîné, il est temps de créer notre porte pour chien pour Bo ! Nous pouvons commencer par examiner les prédictions du modèle. Nous allons prétraiter l'image de la même manière que nous l'avons fait pour notre dernière porte pour chien.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

Essayons avec quelques images :

In [ ]:
make_predictions('data/presidential_doggy_door/valid/bo/bo_20.jpg')

In [ ]:
make_predictions('data/presidential_doggy_door/valid/not_bo/121.jpg')

Ici, il semble qu'un nombre négatif signifie qu'il s'agit de Bo et qu'un nombre positif signifie qu'il s'agit d'autre chose. Nous pouvons utiliser cette information pour que notre porte pour chien ne laisse entrer que Bo !

## Exercice : Créer une porte automatique pour Bo

Complétez le code suivant (FILLME) :

In [ ]:
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if preds[0]< FILLME:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")

Let's try it out!

In [ ]:
presidential_doggy_door('data/presidential_doggy_door/valid/not_bo/131.jpg')

In [ ]:
presidential_doggy_door('data/presidential_doggy_door/valid/bo/bo_29.jpg')

## Summary

Excellent travail ! Avec l'apprentissage par transfert, vous avez construit un modèle très précis à partir d'un très petit ensemble de données. Cette technique peut être extrêmement puissante et faire la différence entre un projet réussi et un projet qui n'arrive pas à décoller. Nous espérons que ces techniques vous aideront à l'avenir dans des situations similaires !